<a href="https://colab.research.google.com/github/RaulMrSouza/Covid_19_Casos_Por_100k_habitantes_no_Estado_de_S-o_Paulo/blob/master/Covid_19_Casos_Por_100k_habitantes_no_Estado_de_S%C3%A3o_Paulo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
#pacotes e imports iniciais
!pip install folium==0.8.2
import folium
from folium import plugins
import pandas as pd
import numpy as np
import json
!pip install requests==v2.21.0
import requests

In [0]:
#trazer json de malha geografica da api do Ibge

headers = {
    'Content-Type': 'application/json;charset=UTF-8', 'User-Agent': 'google-colab',
    'Accept': 'application/json, text/plain, */*', 'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7'
}
cidades = requests.get("https://servicodados.ibge.gov.br/api/v2/malhas/35/?formato=application/vnd.geo+json&resolucao=5", headers=headers)
cidades = cidades.json()

In [10]:
#baixar csv do dataset
url="https://brasil.io/dataset/covid19/caso_full/?state=SP&format=csv"
covid = pd.read_csv(url)
estado = covid[covid.place_type == 'state']
covid = covid[covid.is_last == True]
#desconsiderar estado
covid = covid[covid.place_type != 'state']
#remover nulos
covid = covid[covid.last_available_confirmed_per_100k_inhabitants > 0]
#covid = covid[~covid.last_available_confirmed.isnull()]
estado.head()

,epidemiological_week,date,order_for_place,state,city,city_ibge_code,place_type,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,new_confirmed,last_available_deaths,new_deaths,last_available_death_rate,estimated_population_2019,is_last,is_repeated
550,23,2020-06-03,100,SP,NaN,35.0,state,123483,268.91454,5188,8276,282,0.0670,45919049.0,True,False
1097,23,2020-06-02,99,SP,NaN,35.0,state,118295,257.61640,6999,7994,327,0.0676,45919049.0,False,False
1638,23,2020-06-01,98,SP,NaN,35.0,state,111296,242.37436,1598,7667,52,0.0689,45919049.0,False,False
2179,23,2020-05-31,97,SP,NaN,35.0,state,109698,238.89432,2556,7615,83,0.0694,45919049.0,False,False
2719,22,2020-05-30,96,SP,NaN,35.0,state,107142,233.32800,5586,7532,257,0.0703,45919049.0,False,False


In [0]:
covid['codarea'] = covid['city_ibge_code']
covid = covid.set_index('codarea')
covid['codarea'] = covid.index.astype(int).astype(str)
covid['codarea'] = covid['codarea']

In [12]:
covid = covid.sort_values(by=['last_available_confirmed_per_100k_inhabitants'], ascending=False)
covid[['city', 'last_available_confirmed_per_100k_inhabitants', 'last_available_confirmed', 'last_available_deaths', 'estimated_population_2019']].head(10)

,city,last_available_confirmed_per_100k_inhabitants,last_available_confirmed,last_available_deaths,estimated_population_2019
codarea,,,,,
3524501.0,Jaci,1103.72152,78,2,7067.0
3548500.0,Santos,911.35466,3949,163,433311.0
3547650.0,Santa Salete,841.42395,13,0,1545.0
3550308.0,São Paulo,545.02836,66777,4564,12252023.0
3520202.0,Igaratá,534.92763,51,1,9534.0
3536208.0,Pariquera-Açu,529.31596,104,2,19648.0
3505708.0,Barueri,466.84319,1280,138,274182.0
3548807.0,São Caetano do Sul,445.61123,718,40,161127.0
3534401.0,Osasco,427.68084,2987,336,698418.0


In [0]:
#adicionar campos para layout no mapa para cada cidade
for cidade in cidades['features']: 
  codarea = cidade['properties']['codarea']
  if int(codarea) in covid.index.values:
    cidade['properties']['Cidade'] = str(covid.loc[int(codarea), "city"])
    cidade['properties']['Data Informacao'] = covid.loc[int(codarea), "date"]
    cidade['properties']['Populacao'] = int(covid.loc[int(codarea), "estimated_population_2019"])
    cidade['properties']['Casos por 100k Habitantes'] = int(covid.loc[int(codarea), "last_available_confirmed_per_100k_inhabitants"])
    cidade['properties']['Total de Casos'] = int(covid.loc[int(codarea), "last_available_confirmed"])
    cidade['properties']['Total de mortes'] = int(covid.loc[int(codarea), "last_available_deaths"])
    cidade['properties']['Taxa de mortallidade'] = float(covid.loc[int(codarea), "last_available_death_rate"])
  else:
    cidade['properties']['Cidade'] = 'Sem Dados'
    cidade['properties']['Data Informacao'] = ''
    cidade['properties']['Populacao'] = 0
    cidade['properties']['Casos por 100k Habitantes'] = 0
    cidade['properties']['Total de Casos'] = 0
    cidade['properties']['Total de mortes'] = 0
    cidade['properties']['Taxa de mortallidade'] = 0

In [18]:
#mapa de São Paulo com casos por cidade
m = folium.Map(
    location=[-22,-48],zoom_start=7.5
)

c = folium.Choropleth(
    geo_data = cidades, 
    data = covid,
    columns = ['codarea','last_available_confirmed_per_100k_inhabitants'],
    key_on = 'feature.properties.codarea',
    fill_color = 'YlOrRd', 
    fill_opacity = 0.7, 
    line_opacity = 0.3,
    legend_name = 'Casos Confirmados por 100 k habitantes'
).add_to(m)

c.geojson.add_child(
    folium.features.GeoJsonTooltip(fields=['Cidade', 'Data Informacao', 'Casos por 100k Habitantes', 'Populacao', 'Total de Casos', 'Total de mortes', 'Taxa de mortallidade'], labels=True)
)

folium.LayerControl().add_to(m)

display(m)